Malika Uteuliyeva. Assignment 7. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
import math as m

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
e=m.e

In [3]:
def f(x):
    return (x**10)*(2-x)*(1-e**(20*(x-1)))

In [4]:
integral, error = integrate.quad(f, 0, 1) 
def error(experimental):
    return abs(integral-experimental)/integral

In [5]:
def trapezoidal_rule(x, y, n):
    h = (y-x)/float(n)
    s = (f(x) + f(y))/2
    for i in range(1,n):
        s = s + f(x + i*h)
    return h*s

In [6]:
def romberg_integration(x, y, n):
    result = np.zeros((n, n))
    for i in range(0, n):
        result[i, 0] = trapezoidal_rule(x, y, 2**i)
        for j in range(0, i):
            result[i, j+1] = (result[i, j]*4**(j+1) - result[i-1, j]) / (4**(j+1) - 1)  
    return result[i,n-1]

In [7]:
def adaptive_quadrature(x, y, tolerance):
    z=(x+y)/2
    return quadrature_step(x, y, tolerance, f(x), f(y), f(z))

def quadrature_step(x, y, tolerance, fx, fy, fz):
    h1=y-x
    h2=h1/2
    z=(x+y)/2
    f_s=f((x+z)/2)
    f_t=f((y+z)/2)
    integral1=h1/6*(fx+4*fz+fy)
    integral2=h2/6*(fx+4*f_s+2*fz+4*f_t+fy)
    if abs(integral2-integral1)<=tolerance:
        result=integral2+(integral2-integral1)/15
    else:
        integral_a=quadrature_step(x, z, tolerance, fx, f_s, fz)
        integral_b=quadrature_step(z, y, tolerance, fz, f_t, fy)
        result=integral_a+integral_b
    return result

In [8]:
def GaussLegendreQuadrature(order, tolerance, a, b):
    [Ws,xs, err]= GaussLegendreWeights(order, tolerance)
    integral=(b-a)*0.5*sum( Ws*f( (b-a)*0.5*xs+ (b+a)*0.5 ) )
    return integral

def GaussLegendreWeights(order, tolerance):
    W=[]
    [xis,err]=LegendreRoots(order, tolerance)
    if err==0:
        W=2.0/( (1.0-xis**2)*(DLegendre(order,xis)**2) )
        err=0
    else:
        err=1 
    return [W, xis, err]

def LegendreRoots(order,tolerance):
    if order<2:
        err=1 
    else:
        roots=[]
        for i in range(1,int(order/2 +1)):
            x=m.cos(m.pi*(i-0.25)/(order+0.5))
            error=10*tolerance
            iters=0
            while (error>tolerance) and (iters<1000):
                dx=-Legendre(order,x)/DLegendre(order,x)
                x=x+dx
                iters=iters+1
                error=abs(dx)
            roots.append(x)
        roots=np.array(roots)
        if order%2==0:
            roots=np.concatenate( (-1.0*roots, roots[::-1]) )
        else:
            roots=np.concatenate( (-1.0*roots, [0.0], roots[::-1]) )
        err=0 
    return [roots, err]

def DLegendre(n,x):
    x=np.array(x)
    if (n==0):
        return x*0
    elif (n==1):
        return x*0+1.0
    else:
        return (n/(x**2-1.0))*(x*Legendre(n,x)-Legendre(n-1,x))

def Legendre(n,x):
    x=np.array(x)
    if (n==0):
        return x*0+1.0
    elif (n==1):
        return x
    else:
        return ((2.0*n-1.0)*x*Legendre(n-1,x)-(n-1)*Legendre(n-2,x))/n

The results are the following:

In [9]:
print("The integral value = ", integral)
print ('Error = ', error(integral))

The integral value =  0.06446182039578609
Error =  0.0


In [10]:
trapezoidal_value = trapezoidal_rule(0, 1, 1000000)
print('Integral value by trapezoidal rule = ', trapezoidal_value)
print ('Error = ', error(trapezoidal_value))

Integral value by trapezoidal rule =  0.06446182039411927
Error =  2.5857468033002245e-11


In [11]:
romberg_integral = romberg_integration(0, 1, 15)
print('Integral value by romberg integration = ', romberg_integral)
print('Error = ', error(romberg_integral))

Integral value by romberg integration =  0.0644618203958
Error =  3.44459097745e-15


In [12]:
integral_quadapt = adaptive_quadrature(0, 1, 10**(-10))
print('Integral value by adaptive quadrature = ',integral_quadapt)
print('Error = ',error(integral_quadapt))

Integral value by adaptive quadrature =  0.06446107883605244
Error =  1.1503859634956019e-05


In [13]:
integral_GaussLegendreQuadrature = GaussLegendreQuadrature(10, 10**(-16), 0, 1)
print('Integral value by Gauss quadrature = ', integral_GaussLegendreQuadrature)
print('Error = ',error(integral_GaussLegendreQuadrature))

Integral value by Gauss quadrature =  0.06446210446
Error =  4.40670459556e-06


In [14]:
%timeit trapezoidal_rule(0, 1, 1000000)

768 ms ± 24.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit romberg_integration(0, 1, 15)

25.4 ms ± 816 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit adaptive_quadrature(0, 1, 10**(-13))

9.77 s ± 547 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit GaussLegendreQuadrature(10, 10**(-16), 0, 1)

24.1 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


We can conclude that Romberg integral is the most convenient numerical integration method here as it gives the smallest error (3.44459097745e-15) in the adequately small amount of time (23.8 ms ± 930 µs per loop). One can achieve even better precision of 10**(-16) by using the trapezoidal rule BUT it is timeconsuming and cannot be compared to Romberg in terms of computational cost.


In terms of speed from the fastest to the slowest:

1)adaptive_quadrature
2)romberg_integration
3)GaussLegendreQuadrature
4)trapezoidal_rule.
However, Romberg and Gauss speeds are close, so that the performance and places may differ on another CPU.